# Libraries

In [1]:
import os

# go up one directory
os.chdir("..")

import pandas as pd 
import numpy as np
from tqdm import tqdm
from sklearn_extra.cluster import KMedoids
from functions import cosmic_val
from models.nmf import nmf, refit_NMF
from functions import cosmic_val
from functions import data_handling as dh

# set seed
# np.random.seed(15)

# Data

In [2]:
data_path = "data/catalogues_Ovary_SBS.tsv"
cosmic_path = "data/COSMIC_v3.4_SBS_GRCh37.txt"
output_folder = "data/processed"
output_filename = "Ordered_Ovary_SBS.csv"
ordered_data_path = os.path.join(output_folder, output_filename)

In [3]:
dh.load_preprocess_data(data_path, cosmic_path, sep1 = "\t", sep2 = "\t", output_folder = output_folder, output_filename = output_filename)

Data already exists in  data/processed/Ordered_Ovary_SBS.csv


In [4]:
# load data
data = pd.read_csv(ordered_data_path, index_col = 0)
cosmic = pd.read_csv(cosmic_path, sep = "\t", index_col = 0)

In [5]:
LATENT_DIM = 4
TOLERANCE = 1e-4
MAX_ITERATIONS = 100_000_000
SPLIT = 0.8

In [6]:
losses_train = []
losses_test = []
signatures = []
iterations = 1


for i in tqdm(range(iterations)):


    # Extract train and test data
    train_data = data.sample(frac = SPLIT, axis = 1)
    test_data = data.drop(train_data.columns, axis = 1)
    
    # Applying NMF
    signatures_nmf, exposures_nmf, loss_nmf = nmf(catalog_matrix = train_data.to_numpy(),
                                                  num_sign = LATENT_DIM,
                                                  tol = TOLERANCE,
                                                  max_iter = MAX_ITERATIONS)
    

    # Refitting NMF (test data)

    losses_refit = refit_NMF(test_data.to_numpy(), signatures_nmf)
    losses_test.append(losses_refit[-1])

    # Calculating signatures and exposures for NMF
    diagonals_nmf = signatures_nmf.sum(axis=0)
    exposures_nmf = exposures_nmf.T @ np.diag(diagonals_nmf)
    signatures_nmf = (signatures_nmf) @ np.diag(1 / diagonals_nmf)
    
    losses_train.append(loss_nmf[-1])
    signatures.append(signatures_nmf)



  0%|          | 0/1 [00:00<?, ?it/s]

Iteration: 1000, Loss: 16069.667324872784
Iteration: 2000, Loss: 15522.852279786088
Iteration: 3000, Loss: 13466.912904698695
Iteration: 4000, Loss: 13341.96480095975
Iteration: 5000, Loss: 13323.642686156858
Iteration: 6000, Loss: 13266.956068467742
Iteration: 7000, Loss: 13254.493504144322
Iteration: 8000, Loss: 13026.57161210437
Iteration: 9000, Loss: 13019.97565646557
Iteration: 10000, Loss: 13014.706351306673
Iteration: 11000, Loss: 13008.337430705345
Iteration: 12000, Loss: 13006.195188787231
Iteration: 13000, Loss: 13005.772654343444
Iteration: 14000, Loss: 13005.613552920977
Iteration: 15000, Loss: 13004.8584231583
Iteration: 16000, Loss: 13004.381392278832
Iteration: 17000, Loss: 13002.282632992843
Iteration: 18000, Loss: 13002.014815982107


  0%|          | 0/1 [00:13<?, ?it/s]


UnboundLocalError: local variable 'loss' referenced before assignment

In [ ]:
print("Losses train: ", np.mean(losses_train))
print("Losses test: ", np.mean(losses_test))

In [ ]:
all_signatures = np.hstack(signatures)

In [ ]:
pam = KMedoids(n_clusters = LATENT_DIM, metric='cosine').fit(all_signatures.T)
consensus_signatures = all_signatures[:, pam.medoid_indices_]

In [ ]:
matched_signatures, mean_similarity = cosmic_val.compute_match(consensus_signatures, cosmic, index = 0)

In [ ]:
print(matched_signatures.head())
print("\nMean similarity of the matched signatures: ", mean_similarity)